# Importing numpy lib and text-sample for training

In [1]:
import numpy as np



text = '''Chemical autoencoders are attractive models as they combine chemical space navigation with possibilities for de novo molecule generation in areas of interest.
 This enables them to produce focused chemical libraries around a single lead compound for employment early in a drug discovery project. Here, it is shown that the choice
  of chemical representation, such as strings from the simplified molecular-input line-entry system (SMILES), has a large influence on the properties of the latent space.
   It is further explored to what extent translating between different chemical representations influences the latent space similarity to the SMILES strings or circular fingerprints.
    By employing SMILES enumeration for either the encoder or decoder, it is found that the decoder has the largest influence on the properties of the latent space.
     Training a sequence to sequence heteroencoder based on recurrent neural networks (RNNs) with long short-term memory cells (LSTM) to predict different enumerated SMILES strings
      from the same canonical SMILES string gives the largest similarity between latent space distance and molecular similarity measured as circular fingerprints similarity.
       Using the output from the code layer in quantitative structure activity relationship (QSAR) of five molecular datasets shows that heteroencoder derived vectors markedly
        outperforms autoencoder derived vectors as well as models built using ECFP4 fingerprints, underlining the increased chemical relevance of the latent space.
         However, the use of enumeration during training of the decoder leads to a marked increase in the rate of decoding to different molecules than encoded,
          a tendency that can be counteracted with more complex network architectures.'''

## text is from doi:10.3390/biom8040131

# Some helper-function and generator for training

In [2]:
def preprocess(text):

    bad_symb = '''.,'"{}[]()1234567890!?^:;#@%$~`*><'''
    for i in bad_symb:
        text = text.replace(i,'')

    words = [word.lower() for word in text.split()]
    vocab = dict([(c,i) for i,c in enumerate(sorted(set(words)))])
    print('Preprocessing was finished, bad symbols were deleted, the text was lowercased and the dictionary was made')
    print('Number of words in dictionary: ', len(vocab))

    return words, vocab
    

def generator(words, vocab, window_size, cbow = True):

    vocab_len = len(vocab)
    one_hot = np.eye(vocab_len)
    
    for i in range(len(words)):
        index = i-window_size
        if index < 0:
            index = 0
        context = words[index:i] + words[i+1:i+window_size+1]

        if not cbow:
            yield (one_hot[vocab[words[i]]], vocab[words[i]]), one_hot[[vocab[i] for i in context]]
        else:
            yield one_hot[vocab[words[i]]], one_hot[[vocab[i] for i in context]]



def softmax(vector):    
    return  np.exp(vector) / np.sum(np.exp(vector), axis = 1)

# **The main two classes - implementations of CBOW and SkipGram algorithms.** Note that these implementations are not optimized, i.e. without Negative Sampling, Hierarchical Softmax etc.

In [3]:
class CBOW():
    
    def __init__(self, vocab_size, embedd_size, rate):
        
        self.embedd_size = embedd_size
        self.W1 = np.random.uniform(-0.8, 0.8, (vocab_size, embedd_size))
        self.W2 = np.random.uniform(-0.8, 0.8, (embedd_size, vocab_size))
        self.rate = rate
        
                 
    def forward_pass(self, context):

        self.aver_x = np.mean(context, axis = 0)
        
        self.H = np.dot(self.aver_x, self.W1).reshape(-1,self.embedd_size)
        self.O = np.dot(self.H, self.W2)

        return softmax(self.O)

    
    def get_loss(self, word):
        
        return -self.O[0][np.argwhere(word == 1)].item() + np.log(np.sum(np.exp(self.O)))

    def backward(self, out, word, k = 10):

        error = out - word

        dw2 = np.outer(self.H, error)
        dw1 = np.outer(self.aver_x, np.dot(self.W2, error.T))

        
        self.W2 = self.W2 - self.rate * dw2
        self.W1 = self.W1 - self.rate * dw1

    def get_embedd(self):

        return self.W2





class SKIPGRAM():
    
    def __init__(self, vocab_size, embedd_size, rate):
        
        self.embedd_size = embedd_size
        self.W1 = np.random.uniform(-0.8, 0.8, (vocab_size, embedd_size))
        self.W2 = np.random.uniform(-0.8, 0.8, (embedd_size, vocab_size))
        self.rate = rate
        
                 
    def forward_pass(self, word, idx):

        self.word = word
        self.idx = idx

        self.H = self.W1[self.idx,:].reshape(-1,1)
        self.O = np.dot(self.H.T, self.W2)

        return softmax(self.O)

    
    def get_loss(self, context):

        return -np.sum([self.O[0][np.argwhere(word == 1)] for word in context]) + context.shape[0]*np.log(np.sum(np.exp(self.O)))

    def backward(self, out, context):


        EI = np.sum([np.subtract(out, word) for word in context], axis=0)

        dW2 = np.outer(self.H, EI)
        dW1 = np.outer(self.word, np.dot(self.W2, EI.T))

        self.W2 = self.W2 - self.rate * dW2
        self.W1 = self.W1 - self.rate * dW1

    def get_embedd(self):

        return self.W1           

# **Function for training process**

In [4]:
def train(data, cbow = True, epochs = 500, window_size = 1, embedd_size = 1, rate = 0.01):
    
    words, vocab = preprocess(data)

    if cbow:
        print("Chosen algorithm: CBOW")
        model = CBOW(len(vocab), embedd_size, rate)
    else:
        print("Chosen algorithm: SkipGram")
        model = SKIPGRAM(len(vocab), embedd_size, rate)
    
    
    for epoch in range(1, epochs+1):
        sum_loss = 0
        gener = generator(words, vocab, window_size, cbow = cbow)
        
        for word, context in gener:
            

            if cbow:
                out = model.forward_pass(context)
                sum_loss += model.get_loss(word)

                model.backward(out, word)
            else:
                out = model.forward_pass(word[0], word[1])
                sum_loss += model.get_loss(context)
                
                model.backward(out, context)

        print(f"| EPOCH № {epoch :3.0f} | LOSS {sum_loss / len(vocab):.5f} |")
            
    embeddings = model.get_embedd()

    if cbow:
        n = 0
        for key, value in vocab.items():
            vocab[key] = embeddings[:,n]
            n += 1
    else:
        n = 0
        for key, value in vocab.items():
            vocab[key] = embeddings[n,:]
            n += 1    
    return vocab

In [5]:
embedd_matrix = train(text, cbow = False, epochs = 20, window_size = 4, embedd_size = 3, rate = 0.01)

Preprocessing was finished, bad symbols were deleted, the text was lowercased and the dictionary was made
Number of words in dictionary:  146
Chosen algorithm: SkipGram
| EPOCH №   1 | LOSS 68.92754 |
| EPOCH №   2 | LOSS 68.55180 |
| EPOCH №   3 | LOSS 68.29436 |
| EPOCH №   4 | LOSS 68.09626 |
| EPOCH №   5 | LOSS 67.92907 |
| EPOCH №   6 | LOSS 67.77580 |
| EPOCH №   7 | LOSS 67.62317 |
| EPOCH №   8 | LOSS 67.45744 |
| EPOCH №   9 | LOSS 67.26159 |
| EPOCH №  10 | LOSS 67.01307 |
| EPOCH №  11 | LOSS 66.68449 |
| EPOCH №  12 | LOSS 66.25501 |
| EPOCH №  13 | LOSS 65.74275 |
| EPOCH №  14 | LOSS 65.21697 |
| EPOCH №  15 | LOSS 64.72383 |
| EPOCH №  16 | LOSS 64.25876 |
| EPOCH №  17 | LOSS 63.81201 |
| EPOCH №  18 | LOSS 63.38038 |
| EPOCH №  19 | LOSS 62.96377 |
| EPOCH №  20 | LOSS 62.56252 |


In [25]:
print('{0:<30} {1}'.format('\033[1m'+'WORD:', '\033[1m'+'ITS EMBEDDING:'+'\033[0m'))

for key, value in embedd_matrix.items():
    print('{0:<15} {1}'.format(key, value))


WORD:                      ITS EMBEDDING:
a               [-0.71950402  0.19014475  0.35748951]
activity        [-0.77394885  0.69306238 -0.75172485]
and             [-0.21999558 -0.17156522 -0.13102351]
architectures   [ 0.85103151 -0.27673316 -0.59617004]
are             [-0.34195148  0.35540273  0.15552566]
areas           [-0.6446661   1.01901023  0.58281092]
around          [ 0.27784189 -0.42979749 -0.76278331]
as              [-0.84488166  0.46487471 -0.27227158]
attractive      [-0.11213078 -0.40852608  0.41413694]
autoencoder     [0.04143346 0.10921397 0.66711346]
autoencoders    [-0.09982664  0.40762047 -0.85332113]
based           [0.04207131 0.17024031 0.50096703]
be              [ 0.73039243 -0.18719958  0.24761619]
between         [0.26107065 0.30192845 0.49008755]
built           [-0.71362189  0.08054468 -1.07995724]
by              [0.14222712 0.05850637 0.45264653]
can             [ 0.21010938 -0.53165574 -0.40563127]
canonical       [-0.20661727 -1.03334686  0.53015043